In [6]:
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number
from langchain_openai import ChatOpenAI
from kor import Object, Text, Number  # Assuming kor provides these
from langchain_openai import OpenAI

api_key=''

llm = ChatOpenAI(
    model_name="gpt-4o",
    temperature=0,
    max_tokens=2000,
    api_key=api_key
)

In [7]:
laycanPeriod_schema = Object(
    id="laycanPeriod",
    description="Extracts the laycan period from text, identifying start and end dates for cargo laycan.",
    attributes=[
        Text(
            id="laycanFrom",
            description="The start date of the laycan period in ISO 8601 format."
        ),
        Text(
            id="laycanTo",
            description="The end date of the laycan period in ISO 8601 format."
        )
    ],
    many=False,
    examples=[
        (
            "TOS 4th Quarter 2024",
            [
                {"laycanFrom": "2024-10-01T00:00:00", "laycanTo": "2024-12-31T23:59:59"}
            ]
        ),
        (
            "Laycan: March 2023 to May 2023",
            [
                {"laycanFrom": "2023-03-01T00:00:00", "laycanTo": "2023-05-31T23:59:59"}
            ]
        ),
        (
            "shipment time July 2025",
            [
                {"laycanFrom": "2025-05-01T00:00:00", "laycanTo": "2025-05-31T23:59:59"}
            ]
        ),
        (
            "Ly: 18th to 25th November 2024.",
            [
                {"laycanFrom": "2024-11-18T00:00:00", "laycanTo": "2024-11-25T23:59:59"}
            ]
        ),
        (
            "18/11 - 3/12",
            [
                {"laycanFrom": "2024-11-18T00:00:00", "laycanTo": "2024-12-03T23:59:59"}
            ]
        ),
        (
            "15-25 Feb",
            [
                {"laycanFrom": "2024-02-15T00:00:00", "laycanTo": "2024-02-25T23:59:59"}
            ]
        ),
    ],
)

In [8]:
cargoOrders_schema = Object(
    id="cargoOrders",
    description="Details of cargo orders including freight, laycanPeriod, loading/discharging rates, terms.",
    attributes=[
        Text(
            id="cargo_items",
            description="Names of the cargo items to be transported",
            many=True,
            examples=[
                ("Engines ex Vaasa\nPOL: Vaasa, FI\nCargo: 4511 cbm / 1454 tons / incl 3x 205 mt units\nPOD", ["Engines: 4511 cbm / 1454 tons / incl 3x 205 mt units"]),
            ]
        ),
        Text(
            id="loading_ports",
            description="Name of the ports where the cargo will be loaded",
            many=True,
            examples=[
                ("POL: Hamburg, DE", ["Hamburg"]),
                ("The shipment will be loaded at Bergen and Shanghai.", ["Bergen", "Shanghai"]),
                ("Cargo load at Da Nang", ["Da Nang"])
            ]
        ),
        Text(
            id="discharging_ports",
            description="Name of the ports where the cargo will be dischared",
            many=True,
            examples=[
                ("Cargo Unload at Arzew, Algeria", ["Arzew"]),
                ("Shipment Discharge at Houston or Leer", ["Houston", "Leer"]),
                ("POD: Duqm, Oman", ["Duqm"])
            ]
        ),
        Number(
            id="freight",
            description="The freight cost as a numeric value, e.g., 455000."
        ),
        Text(
            id="currency",
            description="The currency unit for the freight, USD or EUR."
        ),
        Text(
            id="detail_requirements",
            description="Specific vessel requirements, route, ..., if any, e.g."
        ),
        Number(
            id="commission",
            description="Commission as numeric value, in percentage",
            examples=[
                ("5% comm", 5),
                ("commission: 7 %", 7)
            ]
        ),
        laycanPeriod_schema,
    ],
    examples=[
        (
            "The cargo order includes 2 hydraulic power units to be loaded and shipped from Shanghai to Qidong. basis +/- 10% in Charterer, The loading rate is 500 tons/day, and the discharging rate is 600 tons/day. Freight is 455000 USD. Laycan is from 15th Feb to 25th Feb, and arrival is until 06th April agw, wp, fme.",
            [
                {
                    "cargo_items": ["2 hydraulic power units"],
                    "commission": 10,
                    "freight": 455000,
                    "currencyUnit": "USD",
                    "laycanPeriod": {
                        "laycanFrom": "2024-02-15T00:00:00",
                        "laycanTo": "2024-02-25T00:00:00"
                    },
                    "loading_ports": [ "Bergen", "Shanghai" ],
                    "discharging_ports": [ "Qidong" ],
                    "detail_requirements": "Panamax vessel required"
                }
            ]
        )
    ],
    many=True  # Allows for multiple cargo orders
)
chain = create_extraction_chain(llm, cargoOrders_schema, encoder_or_encoder_class="json")
print(chain.get_prompts()[0].format_prompt(text="[user input]").to_string())

Your goal is to extract structured information from the user's input that matches the form described below. When extracting information please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.

```TypeScript

cargoOrders: Array<{ // Details of cargo orders including freight, laycanPeriod, loading/discharging rates, terms.
 cargo_items: Array<string> // Names of the cargo items to be transported
 loading_ports: Array<string> // Name of the ports where the cargo will be loaded
 discharging_ports: Array<string> // Name of the ports where the cargo will be dischared
 freight: number // The freight cost as a numeric value, e.g., 455000.
 currency: string // The currency unit for the freight, USD or EUR.
 detail_requirements: string // Specific vessel requirements, route, ..., if any, e.g.
 commission: number // Commission as numeric value, in percentage
 laycanPeriod: { // Extracts the laycan period from text, identify

In [9]:
testdata = 'POL: Duqm, Oman\nPOD: Arzew, Algeria\nCommodity: Separation and Decarbonation Units\nQty: 9 units\nTotal weight: 1309 MT\nTotal volume: 5380.0332 cbm\nTerms: Requested Full Liner Terms (FLT) as part cargo\nPreference given to proposals with “Waiting for Berth” at both ends at Vessel Owner Account\nLaycan: 18th to 25th November 2024\nSee attached “Scope of Work” document for details\nStowage: Fully Under Deck, lashed and secured\nGrace period: 48 hours reversible POL/POD for loading/discharging\nStackability: Non-Stackable.'

chain.invoke((testdata))["data"]

{'cargoOrders': [{'cargo_items': ['Separation and Decarbonation Units: 9 units, Total weight: 1309 MT, Total volume: 5380.0332 cbm'],
   'loading_ports': ['Duqm'],
   'discharging_ports': ['Arzew'],
   'detail_requirements': 'Requested Full Liner Terms (FLT) as part cargo. Preference given to proposals with “Waiting for Berth” at both ends at Vessel Owner Account. Stowage: Fully Under Deck, lashed and secured. Grace period: 48 hours reversible POL/POD for loading/discharging. Stackability: Non-Stackable.',
   'laycanPeriod': {'laycanFrom': '2024-11-18T00:00:00',
    'laycanTo': '2024-11-25T23:59:59'}}]}

In [10]:
testdata = 'hoffe alles gut :)\nWe can indic asf;\nBBC Rio or sub\nPOL: Bergen\nPOD: Shanghai, Cosco shipyard - Qidong\ncargo:\n1x hydraulic power unit: 13,56 x 4,20 x 2,87m / 44,43mt (as attached/below)\n1x hydraulic power unit: 13,56 x 4,20 x 2,87m / 44,04mt\n+ some spare parts (dets to follow)\nVia Suez\nfreight: USD 455.000 lpsm lthh\nTOS: 15-25 Feb\narrival until 06th April agw, wp, fme\nVia CogH\nfreight: USD 595.000 lpsm lthh\nTOS: 15-25 Feb\narrival until 09th April agw, wp, fme\npart cargo\nunder deck\npartly stackable\notherwise as per BBC BN\nsub further details\nsub costs/restrictions POD\nsub portcaptains approval of stowage'

chain.invoke((testdata))["data"]

{'cargoOrders': [{'cargo_items': ['1x hydraulic power unit: 13,56 x 4,20 x 2,87m / 44,43mt',
    '1x hydraulic power unit: 13,56 x 4,20 x 2,87m / 44,04mt',
    'some spare parts'],
   'loading_ports': ['Bergen'],
   'discharging_ports': ['Shanghai', 'Qidong'],
   'freight': 455000,
   'currency': 'USD',
   'detail_requirements': 'Via Suez, under deck, partly stackable, otherwise as per BBC BN',
   'commission': 0,
   'laycanPeriod': {'laycanFrom': '2024-02-15T00:00:00',
    'laycanTo': '2024-02-25T23:59:59'}},
  {'cargo_items': ['1x hydraulic power unit: 13,56 x 4,20 x 2,87m / 44,43mt',
    '1x hydraulic power unit: 13,56 x 4,20 x 2,87m / 44,04mt',
    'some spare parts'],
   'loading_ports': ['Bergen'],
   'discharging_ports': ['Shanghai', 'Qidong'],
   'freight': 595000,
   'currency': 'USD',
   'detail_requirements': 'Via CogH, under deck, partly stackable, otherwise as per BBC BN',
   'commission': 0,
   'laycanPeriod': {'laycanFrom': '2024-02-15T00:00:00',
    'laycanTo': '2024-02

In [11]:
testdata = 'Good day,\n\nPleased to hear with your indication for:\n\nAccount: DHL Global Forwarding IP\n\nCargo: 1 x motor - total about 179 cbm / about 86 mtons\n\nDimensions: 7,50 x 4,75 x 5,00 m – 86 tons\nAll basis +/- 10% in Charterer\'s option\nOtherwise as per drawings attached hereto\n\nCargo to be loaded under deck\nCargo to be non-stackable / non-overstowable\n\nLoadport: Hamburg or Bremen or Bremerhaven\n\nDischport: Darwin / Australia\n\nLaycan: 04th quarter 2024\n\nTerms: full liner terms hook/hook\nHooking on/off Merchant\'s account\nGrace period of 24hrs/24hrs\n\nTransittime: normal p/c\nPlease advise estimated transittime\n\nContract: BN\n\nCommission: 2.5% ttl on f/d/d/tf\n\nSubjects: Sub all further terms/conditions/details\n\nAwaiting yours in due course, many thanks for your continuous support.\n\nTake care,\nOttmar\n\nOttmar Schulte\nRegional Head of Marine Chartering Europe/Africa\nDGF - Industrial Projects\n\nDHL Global Forwarding GmbH\n\nOffice: +49 40 73355701\nCell: +49 15161578997'

# Assuming chain is already defined and properly set up
chain.invoke(testdata)["data"]


{'cargoOrders': [{'cargo_items': ['1 x motor - total about 179 cbm / about 86 mtons'],
   'loading_ports': ['Hamburg', 'Bremen', 'Bremerhaven'],
   'discharging_ports': ['Darwin'],
   'commission': 2.5,
   'laycanPeriod': {'laycanFrom': '2024-10-01T00:00:00',
    'laycanTo': '2024-12-31T23:59:59'},
   'detail_requirements': "Cargo to be loaded under deck, non-stackable / non-overstowable, full liner terms hook/hook, Hooking on/off Merchant's account, Grace period of 24hrs/24hrs"}]}

In [14]:
testdata = (
    "Hope all is well?\n\nCould you please assist with rates for the following (TOS: 2nd half 2024 only)\n(Sorry for the many different options)\n\nOption A) Engines ex Vaasa\nPOL: Vaasa, FI\nCargo: 4511 cbm / 1454 tons / incl 3x 205 mt units\nPOD: \n1) Los Angeles\n2) Houston\n3) Basis 2 discharge ports (3 engine modules discharge in Houston, remaining cargo in Los Angeles)\n\nOption B) Step up Trafos\nPOL: Vaasa / Izmir\nPOD: Houston\nCargo: 971 cbm / 330 tons / incl 2x 105 mt units\nOption 1) Combination with above engines ex Vaasa\nOption 2) Separate shipment ex Vaasa\nOption 3) Separate shipment ex Turkey\n\nOption C) SCRs / mixing pipes\nCargo: 945 cbm / 77 tons\nPOL: Vaasa combination with above engines\n\nTerms: lthh\nUnder deck / deck option containers\nStackable as per PL\n\nThanks."
)

chain.invoke(testdata)["data"]

{'cargoOrders': [{'cargo_items': ['Engines: 4511 cbm / 1454 tons / incl 3x 205 mt units'],
   'loading_ports': ['Vaasa'],
   'discharging_ports': ['Los Angeles', 'Houston'],
   'laycanPeriod': {'laycanFrom': '2024-07-01T00:00:00',
    'laycanTo': '2024-12-31T23:59:59'},
   'detail_requirements': '3 engine modules discharge in Houston, remaining cargo in Los Angeles'},
  {'cargo_items': ['Step up Trafos: 971 cbm / 330 tons / incl 2x 105 mt units'],
   'loading_ports': ['Vaasa', 'Izmir'],
   'discharging_ports': ['Houston'],
   'laycanPeriod': {'laycanFrom': '2024-07-01T00:00:00',
    'laycanTo': '2024-12-31T23:59:59'},
   'detail_requirements': 'Option 1) Combination with above engines ex Vaasa, Option 2) Separate shipment ex Vaasa, Option 3) Separate shipment ex Turkey'},
  {'cargo_items': ['SCRs / mixing pipes: 945 cbm / 77 tons'],
   'loading_ports': ['Vaasa'],
   'laycanPeriod': {'laycanFrom': '2024-07-01T00:00:00',
    'laycanTo': '2024-12-31T23:59:59'},
   'detail_requirements': '

In [15]:
testdata = (
    "Good day,\n\n"
    "Pleased to hear with your indication for:\n\n"
    "Account: DHL Global Forwarding IP\n\n"
    "Cargo: 1 x motor - total about 179 cbm / about 86 mtons\n\n"
    "Dimensions: 7,50 x 4,75 x 5,00 m – 86 tons\n"
    "All basis +/- 10% in Charterer's option\n"
    "Otherwise as per drawings attached hereto\n\n"
    "Cargo to be loaded under deck\n"
    "Cargo to be non-stackable / non-overstowable\n\n"
    "Loadport: Hamburg or Bremen or Bremerhaven\n\n"
    "Dischport: Darwin / Australia\n\n"
    "Laycan: 04th quarter 2024\n\n"
    "Terms: full liner terms hook/hook\n"
    "Hooking on/off Merchant's account\n"
    "Grace period of 24hrs/24hrs\n\n"
    "Transittime: normal p/c\n"
    "Please advise estimated transittime\n\n"
    "Contract: BN\n\n"
    "Commission: 2.5% ttl on f/d/d/tf\n\n"
    "Subjects: Sub all further terms/conditions/details\n\n"
    "Awaiting yours in due course, many thanks for your continuous support.\n\n"
    "Take care,\n"
    "Ottmar\n\n"
    "Ottmar Schulte\n"
    "Regional Head of Marine Chartering Europe/Africa\n"
    "DGF - Industrial Projects\n\n"
    "DHL Global Forwarding GmbH\n\n"
    "Office: +49 40 73355701\n"
    "Cell: +49 15161578997"
)

chain.invoke(testdata)["data"]


{'cargoOrders': [{'cargo_items': ['1 x motor - total about 179 cbm / about 86 mtons'],
   'loading_ports': ['Hamburg', 'Bremen', 'Bremerhaven'],
   'discharging_ports': ['Darwin'],
   'commission': 2.5,
   'laycanPeriod': {'laycanFrom': '2024-10-01T00:00:00',
    'laycanTo': '2024-12-31T23:59:59'},
   'detail_requirements': "Cargo to be loaded under deck, non-stackable / non-overstowable, full liner terms hook/hook, Hooking on/off Merchant's account, Grace period of 24hrs/24hrs"}]}